<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_10_CNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Lecture_BigData'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Lecture_BigData


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image

# Dataset

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
directory = "petimages-subset/"
train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
validation_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

In [ ]:
import matplotlib.pyplot as plt

class_names = train_dataset.class_names
print(class_names)

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title("{}: {}".format(labels[i].numpy(), class_names[labels[i]]))
        plt.axis("off")

In [5]:
# 正規化
norm_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)

norm_train_dataset = train_dataset.map(lambda x, y: (norm_layer(x), y))
norm_val_dataset = validation_dataset.map(lambda x, y: (norm_layer(x), y))

# VGG16

In [25]:
# データの拡張Layerを定義
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

In [63]:
#--- Layerの準備 ---

IMG_SIZE = (32, 32, 3)

# 入力の前処理
inputs = tf.keras.Input(shape=(None, None, 3))
x = data_augmentation(inputs)
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (IMG_SIZE[0], IMG_SIZE[1])))(x)

In [64]:
""" VGG16 """
x = tf.keras.layers.Lambda(tf.keras.applications.vgg16.preprocess_input)(x)

base_model = tf.keras.applications.vgg16.VGG16(
    input_shape=IMG_SIZE,
    input_tensor=x,
    include_top=False, 
    weights='imagenet'     
)

base_model.trainable = True # base model の重みを固定する場合

# 全結合層出なくGlobal Average Pooling を使用することで計算量を減らせます
GAP_layer = GlobalAveragePooling2D()

# 最終層
pred_layer = Dense(len(class_names), activation='softmax')

#--- モデルの構築 ---
model = tf.keras.Sequential([
    base_model,
    GAP_layer,
    pred_layer
])

In [ ]:
model.summary()

In [ ]:
# (3) モデルのコンパイル
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# (4) モデルの学習開始
model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=3)

In [68]:
# 保存
model_filename = 'mymodel/vgg16'
model.save_weights(model_filename)

In [ ]:
# 読み込み
model.load_weights(model_filename)

In [ ]:
# 再学習
model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=3)

In [71]:
# 保存
model_filename = 'mymodel/vgg16'
model.save_weights(model_filename)

In [ ]:
import numpy as np

for images, labels in validation_dataset.take(1):
    imgs = tf.image.resize(images[:2], (IMG_SIZE[0], IMG_SIZE[1]))
    outputs = model.predict(imgs) # softmax の結果（信頼度）
    predicted = np.argmax(outputs)
    print(outputs)
    print('predicted class label', predicted)
    print('true class label', class_names[labels[i]])

# MobileNet V2

In [58]:
#--- Layerの準備 ---

IMG_SIZE = (96, 96, 3)

# 入力の前処理
inputs = tf.keras.Input(shape=(None, None, 3))
x = data_augmentation(inputs)
x = tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (IMG_SIZE[0], IMG_SIZE[1])))(x)
x = tf.keras.layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)(x)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE,
    input_tensor=x,
    include_top = False,  #False にすることで出力層は読み込みません
    weights='imagenet')

base_model.trainable = True # base model の重みを固定する場合

# 全結合層出なくGlobal Average Pooling を使用することで計算量を減らせます
GAP_layer = GlobalAveragePooling2D()

# 最終層
pred_layer = Dense(len(class_names), activation='softmax')

#--- モデルの構築 ---
model = tf.keras.Sequential([
    base_model,
    GAP_layer,
    pred_layer
])

In [ ]:
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=5)

In [59]:
# 保存
model_filename = 'mymodel/mobilenetv2'
model.save_weights(model_filename)

# ResNet50

モデル構築の部分を少し違った描き方で書いています。

VGGとMobileNetのサンプルでは、層を定義して最後にまとめていますが

以下ResNetのサンプルでは、層を定義すると同時にモデルの構築を行っています。

In [36]:
from tensorflow.keras.layers import Input, Add, Dense, Activation, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50

IMG_SIZE = (224, 224, 3)

def get_model():
    base_model_res50 = ResNet50(
        include_top=True, 
        input_shape=IMG_SIZE,
         weights='imagenet')
    # take the last global average pooling with fewer parameters
    x = base_model_res50.layers[-2].output
    
    x = Dense(2048)(x)
    x = Activation('relu')(x)
    x = Dropout(.5)(x)
    
    x = Dense(2048)(x)
    x = Activation('relu')(x)
    x = Dropout(.5)(x)
    
    x = Dense(10)(x)
    outputs = Activation('softmax')(x)

    model = Model(base_model_res50.input, outputs)
    return model

In [ ]:
model = get_model()
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(norm_train_dataset, validation_data=norm_val_dataset, epochs=5)